## Sentence Transformer

###### pip install -U sentence-transformer

In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
from qdrant_client import models,QdrantClient

In [6]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

## Loading json file called tripod.json

In [7]:
import json 
with open("tripod.json","r",encoding="utf-8") as f:
    documents=json.load(f)
print(type(documents))  # <class 'list'>


<class 'dict'>


In [8]:
client=QdrantClient()

### Creating a collection

In [ ]:
client.create_collection(
    collection_name="daraj_camera",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),#if you’re using all-MiniLM-L6-v2, its output embeddings have 384 dimensions
        distance=models.Distance.COSINE,
    ),
)

True

### Dividing the dataset into the chunks

In [ ]:
import re 
import os
import json

In [ ]:
file_path="tripod.json"
chunk_size=256
base_name=os.path.splitext(os.path.basename(file_path))[0]
# os.path.basename(file_path) file_path=>"/home/yunika/images/camera.jpg" =>camera.jpg
# os.path.splitext(camera.jpg=>camera)
collection_name="camera"
embedding_model_name="all-MiniLM-L6-v2"

### Function for tokenizing the data and then diving into chunks

In [ ]:
def chunk_text(text,chunk_size):
    start=0
    tokens=re.findall(r'\w+|[{}[\]:,",]', text) # tokenize the text and the special characters
    #text = '{"id": 1, "name": "camera"}'
    # tokens = ['{', 'id', ':', '1', ',', 'name', ':', 'camera', '}']
    chunks=[]
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunk = " ".join(tokens[start:end])
        chunks.append(chunk)
        start += chunk_size
# First chunk: ['{', 'id', ':', '1'] → "{ id : 1"
# Second chunk: [',', 'name', ':', 'camera'] → ", name : camera"
# Third chunk: ['}'] → "}"
    return chunks

In [ ]:
# loading the file and convert to dict and json.dumps() converts the dict to String
# JSON file content
# {"name": "Camera", "price": 100}
# # After json.load(f)
# data = {"name": "Camera", "price": 100}  # Python dict

with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)#Reads the content of the file and convert it into a Python object (dict, list,)
raw_text=json.dumps(data)#Converts a Python object back into a JSON-formatted string.

### Calling chunk_text function

In [ ]:
chunks=chunk_text(raw_text,chunk_size=200)
print(len(chunks))

120


In [9]:
client=QdrantClient(url="http://localhost:6333")

In [10]:
encoder.get_sentence_embedding_dimension()

384

In [11]:
payloads={"filename":base_name}

NameError: name 'base_name' is not defined

In [12]:
embeddings=encoder.encode(chunks,show_progress_bar=True,convert_to_numpy=True)

NameError: name 'chunks' is not defined

In [13]:
embeddings

NameError: name 'embeddings' is not defined

In [14]:
client.upsert(
    collection_name="daraj_camera",
    points=[
        models.PointStruct(
            id=idx,
            vector=embeddings[idx].tolist(),
            payload={
                "filename": base_name,     # keep original filename
                "chunk_id": idx,           # which chunk number
                "chunk_text": chunks[idx]  # actual chunk text
            }
            )
            for idx in range(len(chunks)


        )
    ]
)

NameError: name 'chunks' is not defined

In [15]:
query="NeePho NP-3160 Tripod Mobile Phone Camcorder Action Camera 105cm Height"

In [16]:
hits = client.query_points(
    collection_name="daraj_camera",
    query=encoder.encode([query])[0].tolist(),
    limit=5,
).points

for hit in hits:
    print(hit.id, "score:", hit.score)

46 score: 0.43707132
69 score: 0.4156302
0 score: 0.40533966
108 score: 0.40314996
57 score: 0.38794452
